In [305]:
reset()

In [306]:
load('dev.sage')
load('NullBasis.sage')

In [307]:
seed = 1
QUESTION, SOLUTION, CHECKCODE = NullBasis(param=(3,4,2), seed=seed)
HEADING = r"""\centerline{Quiz 1 \hfill MATH 103 / GEAI 1215: Linear Algebra I}"""
KEY = r"""NullBasis %s"""%seed

In [308]:
tex_writer(out_name='Quiz1-1.tex', sol=True, seed=seed, pdf=True)

Quiz1-1.tex has been generated.
Trying to compile the tex file.
The pdf file has been generated.


In [303]:
!ls

 dev.ipynb	  Quiz1-1.aux	   Quiz1-1.tex	   tw_output.log
 dev.sage	  Quiz1-1.log	   Quiz1-1.tex~    tw_output.pdf
 NullBasis.sage  '#Quiz1-1.tex#'   tw_output.aux   tw_output.tex


In [191]:
stg

'CHECKCODE'

In [192]:
CHECKCODE

'8'

In [193]:
eval(stg)

'8'

In [11]:
def create_combiner(out_name, files, pdf=True):
    """Create a tex file for combining
    PDFs in files
    
    Input:
        out_name: string
            name of the output file (including .tex)
        files: list of string
            filenames of the PDFs to be combined
        pdf: boolean
            if True, generate the pdf
    Output:
        No output.  Generate a tex file with filename out_name.  
        Generate the pdf if pdf==True.
    """
    f = open(out_name, 'w')
    f.write(r"""\documentclass{article}

\usepackage{pdfpages}

\begin{document}
""")
    for filename in files:
        f.write(r"""\includepdf{%s}
"""%filename)
    f.write(r"""
\end{document}""")
    
    f.close()

    if pdf:
        os.system('pdflatex %s'%out_name)
        os.system('pdflatex %s'%out_name)
    


In [12]:
!pwd

/home/user/QuizGenerator/factory


In [13]:
all_questions = ['Quiz1-%s.pdf'%j for j in range(1,201)]
all_solutions = ['Quiz1-%s_solution.pdf'%j for j in range(1,201)]

In [14]:
create_combiner(out_name='Quiz1.tex', files=all_questions)

In [298]:
%%writefile NullBasis.sage

def NullBasis(param=(3,4,2), seed=None):
    """A quiz generator asking for the null basis.
    
    Input:
        seed: the random seed
        param: a tuple (m,n,r)
            m,n are the dimensions of matrix A
            r is the rank of A
    Output:
        three strings: QUESTION, SOLUTION, CHECKCODE
    """
    
    m, n, r = param
    random.seed(seed)
    
    A, R, pivots = random_good_matrix(m, n, r, return_answer=True)
    free = [j for j in range(n) if j not in pivots]
    x_var = matrix(n, [var('x_%s'%j) for j in range(1,n+1)])
    k = random.randint(1,n-r)
    
    QUESTION = r"""Consider the equation $\textbf{A}\textbf{x} = \textbf{0}$,
where 
\[\textbf{A}=
%s,
\textbf{x}=
%s,\text{ and }
\textbf{0}=
%s.\]
Compute the reduced echelon form $\textbf{R}$ of $\textbf{A}$ 
to get the free variables.  
Let $k=%s$.  
Find a solution $\textbf{x}=\bm{\beta}_k$ by setting 
the $k$-th free variable as $1$ while  
the other free variables as $0$.

\bigskip
Check code $=$ (sum of all entries of $\bm{\beta}_k$) mod $10$
"""%(latex_matrix(A), 
           latex_matrix(x_var), 
           latex_matrix(zero_matrix(n,1)), 
           k
          )
    
    free_text = ','.join(['x_%s'%(j+1) for j in free])
    xk = 'x_%s'%(free[k-1]+1)
    betak = betak_solver(R, free, k)
    checkcode = sum(betak.transpose()[0]) % 10
    CHECKCODE = "%s"%checkcode
    
    SOLUTION = r"""Apply Gaussian elimination to \textbf{A} 
to get its reduced echelon form 
\[\textbf{R} = 
%s.\]
The free variables are $%s$.  

By setting $%s=1$ and all other free variables as $0$, one may solve for 
\[\rboxed{\bm{\beta}_%s = 
%s
}.\]

as the answer.

\bigskip
Check code $=$ (sum of all entries of $\bm{\beta}_%s$) mod $10$ $= %s$. 
"""%(latex_matrix(R), 
     free_text, 
     xk, 
     k, 
     latex_matrix(betak), 
     k, 
     CHECKCODE
    )
    
    return QUESTION, SOLUTION, CHECKCODE

Overwriting NullBasis.sage


In [15]:
%%writefile dev.sage

import random
import os

def tex_writer(out_name='tw_output.tex', sol=True, template='basic', seed=None, pdf=False):
    """This function takes 
    ../template-*.tex 
    and replace the placeholders by 
    the corresponding strings.
    
    Note: This function requires the following globle variables
        QUESTION, SOLUTION, CHECKCODE, HEADING, KEY
        
    Input:
        out_name: a string for the output filename
        template: 'basic' or 'plain'
        seed: random seed
        pdf: boolean
            generate the pdf if True
    Output:
        No output.  This function only generate the file."""
    
    f = open('../template-%s.tex'%template, 'r')
    g = open(out_name, 'w')
    if sol:
        all_filler = ['QUESTION', 'SOLUTION', 'CHECKCODE', 'HEADING', 'KEY']
    else:
        all_filler = ['QUESTION', 'HEADING', 'KEY']
    for line in f:
        for filler in all_filler:
            if line == "%%%%%{}\n".format(filler):
                g.write(eval(filler))
                break
        else:
            g.write(line)
    f.close()
    g.close()
    
    print out_name + " has been generated."
    
    if pdf:
        print "Trying to compile the tex file."
        os.system('pdflatex %s'%out_name)
        os.system('pdflatex %s'%out_name)
        print "The pdf file has been generated."

def create_combiner(out_name, files, pdf=True):
    """Create a tex file for combining
    PDFs in files
    
    Input:
        out_name: string
            name of the output file (including .tex)
        files: list of string
            filenames of the PDFs to be combined
        pdf: boolean
            if True, generate the pdf
    Output:
        No output.  Generate a tex file with filename out_name.  
        Generate the pdf if pdf==True.
    """
    f = open(out_name, 'w')
    f.write(r"""\documentclass{article}

\usepackage{pdfpages}

\begin{document}
""")
    for filename in files:
        f.write(r"""\includepdf{%s}
"""%filename)
    f.write(r"""
\end{document}""")
    
    f.close()

    if pdf:
        os.system('pdflatex %s'%out_name)
        os.system('pdflatex %s'%out_name)
            
### modified from minrank_aux/general_Lib.sage
def latex_matrix(A):
    m,n=A.dimensions();
    stg = r"""\begin{bmatrix}
"""
    for i in range(m):
        for j in range(n-1):
            stg += r""" %s &"""%A[i][j]
        stg += r""" %s \\ 
"""%A[i][n-1]
    stg += r"""\end{bmatrix}"""
    return stg

def random_ref(m, n, r, bound=5, return_pivots=False):
    """A generator of 
    random reduced echelon form 
    (where the pivots are 1)
    
    Input:
        m, n: integers
        r: positive integer with r <= m and r <= n
        bound: positive integer
            |matrix entries| <= bound
        return_pivots: boolean
            if True, return the pivots also
    Output:
        an m x n matrix with rank r
        in its reduced echelon form
        whose pivots are 1.
        Note: the first pivot always locates at (0,0).
    """
    R = zero_matrix(m,n)
    all_numbers = list(range(-bound, bound+1))
    all_choices = list(Combinations(list(range(1,n)),r-1))
    pivots = [0] + random.choice(all_choices)
    pivots += [n]
    for i in range(r):
        j = pivots[i]
        R[i,j] = 1
        for ii in range(i+1):
            for jj in range(j+1, pivots[i+1]):
                R[ii,jj] = random.choice(all_numbers)
    R = R.change_ring(QQ)
    pivots.remove(n)
    if return_pivots:
        return R, pivots
    else:
        return R

def random_good_matrix(m, n, r, bound=5, return_answer=False):
    """A generator of 
    matrix whose Gaussian elimination is easy.
    
    Input:
        m, n: integers
        r: positive integer with r <= m and r <= n
        bound: positive integer
            |coeff for row operation| <= bound
        return_answer: boolean
            if True, return the reduced echelon form and the pivots also.
    Output:
        an m x n matrix with rank r
        whose Gaussian elimination is easy.
    """
    R, pivots = random_ref(m, n, r, return_pivots=True)
    A = copy(R)
    pivots_rev = copy(pivots)
    pivots_rev.reverse()
    all_numbers = list(range(-bound, bound+1))
    ### upward mix 
    for i in range(r):
        j = pivots_rev[i]
        for ii in range(i):
            A[ii,:] += A[i,:] * random.choice(all_numbers)
    ### downward mix
    for i in range(r):
        j =  pivots_rev[i]
        for ii in range(i+1,m):
            A[ii,:] += A[i,:] * random.choice(all_numbers)
    if return_answer:
        return A, R, pivots
    else:
        return A
    
def find_pivots(ref):
    """Find the pivots of a reduced echelon form.
    
    Input:
        ref: a matrix in reduced echelon form
    Outpu: 
        a list of of indices of the pivots
    """
    m,n = ref.dimensions()
    pivots = []
    for i in range(m):
        for j in range(n):
            if ref[i,j] != 0:
                pivots.append(j)
                break
        else:
            break    
    return pivots

def betak_solver(ref, free, k):
    """Return the solution for 
    ref * x = 0
    whereas the k-th (1-indexing) free variable is 1
    while all other free variables are 0.
    
    Input:
        ref: matrix in reduced echelon form
        free: a list of indices of the free variables
        k: positive integer
    Output:
        A column vector beta_k (n x 1 matrix)
        that satisfies ref * beta_k = 0
        whereas the k-th (1-indexing) free variable is 1
        while all other free variables are 0.
    """
    m,n = ref.dimensions()
    pivots = [j for j in range(n) if j not in free]
    rank = len(pivots)
    betak = [0] * n
    ### k is 1-indexing
    ### free is 0-indexing
    betak[free[k-1]] = 1
    for i in range(rank-1,-1,-1):
        j = pivots[i]
        betak[j] = -sum([ref[i,jj]*betak[jj] for jj in range(j+1,n)])
    return  matrix(n,betak)

Overwriting dev.sage


In [135]:
!cat dev.sage


import random

### modified from minrank_aux/general_Lib.sage
def latex_matrix(A):
    m,n=A.dimensions();
    stg = r"""\begin{bmatrix}
"""
    for i in range(m):
        for j in range(n-1):
            stg += r""" %s &"""%A[i][j]
        stg += r""" %s \\ 
"""%A[i][n-1]
    stg += r"""\end{bmatrix}
    """
    return stg

def random_ref(m, n, r, bound=5, return_pivots=False):
    """A generator of 
    random reduced echelon form 
    (where the pivots are 1)
    
    Input:
        m, n: integers
        r: positive integer with r <= m and r <= n
        bound: positive integer
            |matrix entries| <= bound
        return_pivots: boolean
            if True, return the pivots also
    Output:
        an m x n matrix with rank r
        in its reduced echelon form
        whose pivots are 1.
        Note: the first pivot always locates at (0,0).
    """
    R = zero_matrix(m,n)
    all_numbers = list(range(-bound, bound+1))
    all_choices = list(Combinations(list(range(1,n